# car_inspections & auctions

Aim: Predict car price & returned rate

Original files (2): (FCG...Auctions.csv) & (FCG...Inspections.csv)

[Preprocessing]
(FCG..Inspections.csv) : 2 columns about the score of the used car, in json object
--> This is saved as pandas, then save it as report_cars.csv, inspect_only.csv

[Merging files]
Merge all csv files with join (car_id is overlapping for all csv file)


# load data & libraries

In [1]:
import os
import pandas as pd
import matplotlib.pyplot as plt
# from sklearn.linear_model import LogisticRegression
import numpy as np
# from sklearn.linear_model import Lasso
# from sklearn.model_selection import train_test_split
import json

In [2]:
# file name
auction_fname = 'FCG Germany GmbH_Data Scientist_case study_Auctions.csv'
inspection_fname = 'FCG Germany GmbH_Data Scientist_case study_Inspections.csv'

In [3]:
auction_data = pd.read_csv(auction_fname)
inspection_data = pd.read_csv(inspection_fname)

## Feature engineering

# Inspection extract

inspection_data['category_scores'] (json) ==> inspect_cars (pandas)

In [4]:
# Preassign inspect_allcar
inspect_allcar_pd = pd.DataFrame()

In [84]:
# Extract the first car_id's inspection score
# --> assign as standard name
inspect_car1 = json.loads(inspection_data['category_scores'][0])
pd_car1 = pd.DataFrame.from_dict(inspect_car1)
stand_inspect = pd_car1.id

In [85]:
# Extract for all car
# each car
for i in range(len(inspection_data)):
    inspect_onecar=json.loads(inspection_data['category_scores'][i])
    inspect_onecar_pd=pd.DataFrame.from_dict(inspect_onecar)
    
    # each properties of the car
    for s in range(len(stand_inspect)):
        # run only when inspection feature exists, some car has no inspection
        if len(inspect_onecar_pd)>1: 
            if inspect_onecar_pd['id'][s] == stand_inspect[s]: # when feature name match with standard name
                inspect_allcar_pd.at[i, stand_inspect[s]] = inspect_onecar_pd.score[s]
        else: # when inspection is missing assign NaN
            inspect_allcar_pd.at[i, stand_inspect[s]] = 'NaN'

Add car_id column
(This is to later merge with other files: car_id has to match with other files)

In [74]:
inspect_cars = inspect_allcar_pd.copy()

In [82]:
inspect_cars['car_id'] = inspection_data['car_id']

In [76]:
cols = inspect_cars.columns.tolist()
cols = cols[-1:] + cols[:-1]
inspect_cars = inspect_cars[cols]

Save inspect_cars

In [87]:
inspect_cars.to_csv('inspect_cars.csv', encoding = 'utf-8', index=False)

## Report extract

In [55]:
# Preassign inspect_allcar
report_allcar_pd = pd.DataFrame()

In [56]:
# Extract the first car_id's inspection score
# --> assign as standard name
report_car1 = json.loads(inspection_data['inspection_report'][0])
pd_car1 = pd.DataFrame(list(report_car1.items()))
stand_report = pd_car1.loc[:,0]

In [62]:
# Extract for all car
# each car
for i in range(len(inspection_data)):
    report_onecar=json.loads(inspection_data['inspection_report'][i])
    report_onecar_pd=pd.DataFrame.from_dict(report_onecar, orient = 'index')
    
    # each properties of the car
    for s in range(len(stand_report)):
        if len(report_onecar_pd) >1:#== len(stand_report): # run when there is a inspection feature
            if len(report_onecar_pd.loc[:,0].keys()) > s:
                if report_onecar_pd.loc[:,0].keys()[s] == stand_report[s]: # when feature name match
                    report_allcar_pd.at[i, stand_report[s]] = report_onecar_pd.values[s][0]
                else:
                    report_allcar_pd.at[i, stand_report[s]] = 'NaN'  
            else:
                report_allcar_pd.at[i, stand_report[s]] = 'NaN'
        else: # if feature part is empty
            report_allcar_pd.at[i, stand_report[s]] = 'NaN'

In [63]:
report_allcar_pd.shape, inspection_data.shape

((21160, 190), (21160, 10))

In [66]:
report_cars = report_allcar_pd.copy()

In [68]:
# add car_id
report_cars['car_id'] = inspection_data['car_id']

In [71]:
cols = report_cars.columns.tolist()
cols = cols[-1:] + cols[:-1]
report_cars = report_cars[cols]

In [73]:
## save report_cars
report_cars.to_csv('report_cars.csv', encoding = 'utf-8', index=False)

## Trying to merge two files
not working yet

In [4]:
# Merge the two dataframes, using _ID column as key
df3 = pd.merge(auction_data, inspection_data, on = 'car_id')
# df3.set_index('car_id', inplace = True)

In [5]:
df3.shape, auction_data.shape, inspection_data.shape

((24043, 26), (24053, 17), (21160, 10))

## Extract necessary features & make it X, Y

In [14]:
XY_names =[
# 'auction_created_date', 'auction_start_date','auction_original_end_date','auction_end_date',\
'buy_now_enabled','auction_start_price', 'winning_bid_price',\
'auction_buy_now_price','won_with_buy_now','number_of_extensions',\
'has_winner', 'number_of_bids', 'number_of_bidders','returned']
Y_names = ['returned']
# Y = auction_data['returned']

In [16]:
XY = pd.DataFrame(df3, columns=XY_names)

## Clean Data

In [ ]:
# drop nan & duplicate
XY_dropna = XY.dropna()
XY_dropduplicate = XY_dropna.drop_duplicates()

In [ ]:
DF = XY_dropduplicate.reset_index()

In [ ]:
Xc = DF.iloc[:,0:-1]
Yc = DF.iloc[:,-1]

## Split train & test

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(Xc, Yc, test_size=0.2)

In [ ]:
DF_allcorr = DF.corr()

In [ ]:
corr_result = DF_allcorr['returned'].sort_values(ascending=False)


In [ ]:
plt.figure()
corr_result[1:].plot(kind='bar')


## Machine learning training & implementation

In [ ]:
lasso = Lasso(alpha=0.01)#.fit()#Xc,Yc)

In [ ]:
lasso.fit(X_train,y_train)

In [ ]:
y_pred = lasso.predict(Xc)

In [ ]:
plt.plot(y_pred, '.')
plt.plot(Yc, '.')
plt.show()

In [ ]:
mse = np.mean((y_pred - Yc)**2)
mse


In [ ]:
log_reg = LogisticRegression()
log_reg.fit(X_train, y_train)


In [ ]:
prediction = log_reg.predict(X_test)

In [ ]:
from sklearn.metrics import classification_report

In [ ]:
classification_report(y_test, prediction)

## Basic Correlation

In [ ]:
DF.loc[:,'returned'].value_counts()

In [ ]:
plt.subplot(1,3,1)
plt.hist(DF.loc[:,'auction_start_price'])
plt.subplot(1,3,2)
plt.hist(DF.loc[:,'winning_bid_price'])
plt.subplot(1,3,3)
plt.hist(DF.loc[:,'auction_buy_now_price'])
plt.show()

In [ ]:
# Car Price Prediction

In [ ]:
Car_Price = DF['auction_buy_now_price']

In [ ]:
import csv
from collections import OrderedDict

In [ ]:
with open(auction_fname, 'r') as f:
    r = f.read()
    dict2 = {row[0]: row[1:] for row in r}